# Session 6: LINQ and Extension Methods

<img align="right" width="150" height="113" src="img/csharp_savetheday.png">

[LINQ (Language Integrated Query)](https://docs.microsoft.com/en-us/dotnet/csharp/programming-guide/concepts/linq/?WT.mc_id=visualstudio-twitch-jefritz) is a collection of methods and language features that allow you to interact with collections of data.  In our last session, we focused on **LINQ to Objects** which allows us to use method predicates to interact with those collections.

Let's setup our `Card` class and `FritzSet` collection object to work with again in this workbook

In [1]:
class Card {
    public Card(string def) {
        var values = def.Split('-');
        Rank = values[0];
        Suit = values[1];
    }
    public string Rank;
    public int RankValue { 
        get { 
            var faceCards = new Dictionary<string,int> { {"J", 11}, {"Q", 12}, {"K", 13}, {"A", 14} };
            return faceCards.ContainsKey(Rank) ? faceCards[Rank] : int.Parse(Rank); 
        }
    }
    public string Suit;
    public override string ToString() {
        return $"{Rank}-{Suit}";
    }
    public static implicit operator Card(string id) {
        return new Card(id);
    }
}

class FritzSet<T> : IEnumerable<T> {

    private List<T> _Inner = new List<T>();

    public IEnumerator<T> GetEnumerator()
    {
        return _Inner.GetEnumerator();
    }

    IEnumerator IEnumerable.GetEnumerator()
    {
        return _Inner.GetEnumerator();
    }

    public void Add(T newItem) {
        var insertAt = _Inner.Count == 0 ? 0 : new Random().Next(0,_Inner.Count+1);
        _Inner.Insert(insertAt, newItem);
    }
    
    public FritzSet<T> Shuffle() {
        _Inner = _Inner.OrderBy(_ => Guid.NewGuid()).ToList();
        return this;
    }
    
}

var TheDeck = new FritzSet<Card>();
TheDeck.Add("A-c");TheDeck.Add("A-d");TheDeck.Add("A-h");TheDeck.Add("A-s");TheDeck.Add("2-c");TheDeck.Add("2-d");TheDeck.Add("2-h");TheDeck.Add("2-s");TheDeck.Add("3-c");TheDeck.Add("3-d");TheDeck.Add("3-h");TheDeck.Add("3-s");TheDeck.Add("4-c");TheDeck.Add("4-d");TheDeck.Add("4-h");TheDeck.Add("4-s");
TheDeck.Add("5-c");TheDeck.Add("5-d");TheDeck.Add("5-h");TheDeck.Add("5-s");TheDeck.Add("6-c");TheDeck.Add("6-d");TheDeck.Add("6-h");TheDeck.Add("6-s");TheDeck.Add("7-c");TheDeck.Add("7-d");TheDeck.Add("7-h");TheDeck.Add("7-s");TheDeck.Add("8-c");TheDeck.Add("8-d");TheDeck.Add("8-h");TheDeck.Add("8-s");
TheDeck.Add("9-c");TheDeck.Add("9-d");TheDeck.Add("9-h");TheDeck.Add("9-s");TheDeck.Add("10-c");TheDeck.Add("10-d");TheDeck.Add("10-h");TheDeck.Add("10-s");TheDeck.Add("J-c");TheDeck.Add("J-d");TheDeck.Add("J-h");TheDeck.Add("J-s");
TheDeck.Add("Q-c");TheDeck.Add("Q-d");TheDeck.Add("Q-h");TheDeck.Add("Q-s");TheDeck.Add("K-c");TheDeck.Add("K-d");TheDeck.Add("K-h");TheDeck.Add("K-s");

// TheDeck
TheDeck.Shuffle().Shuffle().Shuffle().Shuffle().Shuffle();
//TheDeck

The below script needs to be able to find the current output cell; this is an easy method to get it.

In review, we can write a little bit of code to work with this collection to deal cards appropriately for a Texas Hold 'em poker game:

In [2]:
var ourDeck = TheDeck.Shuffle().Shuffle();

var hand1 = new List<Card>();
var hand2 = new List<Card>();
var hand3 = new List<Card>();
hand1.Add(ourDeck.Skip(1).First());
hand2.Add(ourDeck.Skip(2).First());
hand3.Add(ourDeck.Skip(3).First());
hand1.Add(ourDeck.Skip(4).First());
hand2.Add(ourDeck.Skip(5).First());
hand3.Add(ourDeck.Skip(6).First());

display("Hand 1");
display(hand1);
display("Hand 2");
display(hand2);
display("Hand 2");
display(hand3);

// Burn a card and deal the next 3 cards called 'the flop'
display("The Flop");
display(ourDeck.Skip(8).Take(3));
    
// Burn a card and take one card called 'the turn'
display("The Turn");
display(ourDeck.Skip(10).First());

// Burn a card and take the final card called 'the river'
display("The River");
display(ourDeck.Skip(12).First());

Hand 1

index,RankValue,Rank,Suit
0,5,5,c
1,9,9,c


Hand 2

index,RankValue,Rank,Suit
0,10,10,h
1,12,Q,h


Hand 2

...

index,RankValue,Rank,Suit
0,13,K,h
1,14,A,c
2,2,2,h


The Turn

RankValue,Rank,Suit
2,2,h


The River

RankValue,Rank,Suit
7,7,d


## Language Integrated Query

You can build [expressions](https://docs.microsoft.com/dotnet/csharp/linq/query-expression-basics?WT.mc_id=visualstudio-twitch-jefritz#what-is-a-query-and-what-does-it-do) in the middle of your C# code that _LOOKS_ like SQL turned sideways.  Query Expressions begin with a `from` clause and there's also a mandatory `select` clause to specify the values to return.  By convention, many C# developers who use this syntax align the clauses to the right of the `=` symbol.  Let's dig into that syntax a bit more:

In [3]:
// The simplest query
var outValues = from card in TheDeck // the required collection we are querying
                select card;         // the values to be returned
                
outValues

index,RankValue,Rank,Suit
0,10,10,d
1,5,5,c
2,10,10,h
3,6,6,c
4,9,9,c
5,12,Q,h
6,12,Q,d
7,13,K,c
8,13,K,h
9,14,A,c


### Where and OrderBy clauses

That's a boring and non-productive query.  You can start to make queries more interesting by adding a `where` clause with an appropriate test in a format similar to that you would find in an `if` statement.  You can also optionally add an `orderby` clause with an **ALSO** optional `descending` keyword.  Tinker with the query in the next block to learn more about these clauses

In [4]:
var results = from card in TheDeck
              where card.Suit == "h"    // Return just the Hearts
              // orderby card.RankValue 
              select card;
              
results

index,RankValue,Rank,Suit
0,10,10,h
1,12,Q,h
2,13,K,h
3,2,2,h
4,11,J,h
5,3,3,h
6,9,9,h
7,8,8,h
8,14,A,h
9,4,4,h


Additionally, nothing is requiring you to return the object in the collection.  You can return different properties and values by changing up the `select` clause:

In [5]:
var results = from card in TheDeck
              where card.Suit == "h" && card.RankValue > 10
              select card.Rank;
              
results              

index,value
0,Q
1,K
2,J
3,A


### Grouping data with the Group clause

Data in your query can be grouped together using the [group clause](https://docs.microsoft.com/dotnet/csharp/language-reference/keywords/group-clause?WT.mc_id=visualstudio-twitch-jefritz).  The `group` clause can be used in place of the `select` clause or can be used with the `select` clause to aggregate data in various groupings.  Let's try using the `group` keywords

In [6]:
var results = from card in TheDeck
              group card by card.Suit;

display(results.GetType());
results

System.Linq.GroupedEnumerable<Submission#4+Card,System.String>

index,value
0,"[ 10-d, Q-d, 7-d, 5-d, 2-d, 6-d, 4-d, 8-d, J-d, 9-d, K-d, 3-d, A-d ]"
1,"[ 5-c, 6-c, 9-c, K-c, A-c, J-c, 3-c, 2-c, 7-c, 10-c, 8-c, 4-c, Q-c ]"
2,"[ 10-h, Q-h, K-h, 2-h, J-h, 3-h, 9-h, 8-h, A-h, 4-h, 6-h, 5-h, 7-h ]"
3,"[ A-s, 5-s, 9-s, 10-s, J-s, 6-s, 4-s, K-s, 8-s, 3-s, 7-s, Q-s, 2-s ]"


Interestingly, we are returned a collection with all of the cards grouped by their suits.  If we also wanted to select the suit and create a grouped result we could expand our query like this:

In [7]:
var results = from card in TheDeck
              group card by card.Suit into suit
              select new {TheSuit=suit.Key, suit};

results

index,TheSuit,suit
0,d,"[ 10-d, Q-d, 7-d, 5-d, 2-d, 6-d, 4-d, 8-d, J-d, 9-d, K-d, 3-d, A-d ]"
1,c,"[ 5-c, 6-c, 9-c, K-c, A-c, J-c, 3-c, 2-c, 7-c, 10-c, 8-c, 4-c, Q-c ]"
2,h,"[ 10-h, Q-h, K-h, 2-h, J-h, 3-h, 9-h, 8-h, A-h, 4-h, 6-h, 5-h, 7-h ]"
3,s,"[ A-s, 5-s, 9-s, 10-s, J-s, 6-s, 4-s, K-s, 8-s, 3-s, 7-s, Q-s, 2-s ]"


<img src="img/facecards.png" align="right">

Now this is **VERY INTERESTING** we have created an [Anonymous Type](https://docs.microsoft.com/dotnet/csharp/programming-guide/classes-and-structs/anonymous-types?WT.mc_id=visualstudio-twitch-jefritz), a type on the fly that contains a string field for `TheSuit` and a collection of `Card` objects in a field called `suit`.  We'll get more into **Anonymous Types** next week, but you need to know that you can use the `new` keyword with curly braces `{ }` to create a type and make it available in your code.  Many C# veterans will recommend against exposing the anonymous type outside of the method it is created in and instead suggest creating a concrete type to return in that `select` clause.

Our groupings can take some interesting calculations.  Let's write a grouping for all of the face cards (and the Ace too):

In [8]:
var results = from card in TheDeck
              group card by card.RankValue > 10 into facecards
              select new {TheSuit=facecards.Key, facecards};

results

index,TheSuit,facecards
0,False,"[ 10-d, 5-c, 10-h, 6-c, 9-c, 2-h, 7-d, 5-d, 5-s, 2-d, 9-s, 6-d, 10-s, 3-h, 9-h, 4-d, 3-c, 8-d, 2-c, 8-h ... (16 more) ]"
1,True,"[ Q-h, Q-d, K-c, K-h, A-c, A-s, J-c, J-s, J-h, J-d, A-h, K-s, Q-c, K-d, Q-s, A-d ]"


That looks strange, but we have two groups:  1 group that are the numeric cards and a second group that are the face cards.  Let's tinker with that method a little more:

In [9]:
var results = from card in TheDeck
              where card.RankValue > 10
              group card by card.Rank into facecards
              select new {Face=facecards.Key, facecards};

results

index,Face,facecards
0,Q,"[ Q-h, Q-d, Q-c, Q-s ]"
1,K,"[ K-c, K-h, K-s, K-d ]"
2,A,"[ A-c, A-s, A-h, A-d ]"
3,J,"[ J-c, J-s, J-h, J-d ]"


Now this sets up for a simplified **Sam the Bellhop** classic card trick.  Take a few minutes and enjoy magician and former Philadelphia Eagles player [Jon Dorenbos performing this trick](https://www.youtube.com/watch?v=fwKPDrtgXRs) where he sorts and finds cards while telling the story of Sam the Bellhop.

## Loading data from CSV

We've worked with objects and data that we've specified here in the notebook. Let's use an external library, in .NET we call them **NuGet Packages** from www.nuget.org called [LINQtoCSV](https://www.nuget.org/packages/LinqToCsv/) to load US Domestic Box Office Data for 2019 (courtesy of [Box Office Mojo](https://www.boxofficemojo.com/year/2019/)).

In [23]:
#r "nuget:LinqToCsv"
using LINQtoCSV;

class MyDataRow { 
    [CsvColumn(Name = "Rank", FieldIndex = 1)]
    public int Rank {get; set;}
    [CsvColumn(Name = "Release", FieldIndex = 2)]
    public string Title { get; set;}
    [CsvColumn(Name = "Theaters", FieldIndex = 3)]
    public int Theaters { get; set;}
    [CsvColumn(Name = "Total Gross", FieldIndex = 4)]
    public long GrossBoxOffice { get; set;}
    [CsvColumn(Name = "Release Date", FieldIndex = 5, OutputFormat="MM/d/yyyy")]
    public DateTime RawReleaseDate { get; set; }
    public DateTime ReleaseDate { get { return new DateTime(2019, RawReleaseDate.Month, RawReleaseDate.Day); } }
    [CsvColumn(Name="Distributor", FieldIndex = 6)]
    public string Distributor { get; set; }
}
var inputFileDescription = new CsvFileDescription
{
    SeparatorChar = ',', 
    FirstLineHasColumnNames = true
};
var context = new CsvContext();
var boxOffice = context.Read<MyDataRow>("data/2019_US_BoxOffice.csv", inputFileDescription);
boxOffice

index,Rank,Title,Theaters,GrossBoxOffice,RawReleaseDate,ReleaseDate,Distributor
0,1,Avengers: Endgame,4662,858373000,2020-04-26 00:00:00Z,2019-04-26 00:00:00Z,Disney
1,2,The Lion King,4802,543638043,2020-07-19 00:00:00Z,2019-07-19 00:00:00Z,Disney
2,3,Toy Story 4,4575,434038008,2020-06-21 00:00:00Z,2019-06-21 00:00:00Z,Disney
3,4,Frozen II,4440,477373578,2020-11-22 00:00:00Z,2019-11-22 00:00:00Z,Disney
4,5,Captain Marvel,4310,426829839,2020-03-08 00:00:00Z,2019-03-08 00:00:00Z,Disney
5,6,Star Wars: Episode IX - The Rise of Skywalker,4406,515202542,2020-12-20 00:00:00Z,2019-12-20 00:00:00Z,Disney
6,7,Spider-Man: Far from Home,4634,390532085,2020-07-02 00:00:00Z,2019-07-02 00:00:00Z,Sony Pictures Entertainment (SPE)
7,8,Aladdin,4476,355559216,2020-05-24 00:00:00Z,2019-05-24 00:00:00Z,Disney
8,9,Joker,4374,335451311,2020-10-04 00:00:00Z,2019-10-04 00:00:00Z,Warner Bros.
9,10,It Chapter Two,4570,211593228,2020-09-06 00:00:00Z,2019-09-06 00:00:00Z,Warner Bros.


In [38]:
// Let's return the total receipts for the top 100 films, grouped by studio 
// with their average number of theaters reported and their top film for 2019

var results = from z in (
                from film in boxOffice
                where film.Rank <= 100
                group film by film.Distributor into dist
                select new { Distributor=dist.Key, 
                    TotalBoxOffice=(
                        from f in dist
                        select f.GrossBoxOffice
                    ).Sum(),
                    AvgTheaters=Math.Round((
                        from f in dist
                        select f.Theaters
                    ).Average()),
                    BestFilm = (
                        from f in dist
                        orderby f.GrossBoxOffice descending
                        select f.Title
                    ).First()
                }
            )
            orderby z.TotalBoxOffice descending
            select new {z.Distributor, TotalBoxOffice=z.TotalBoxOffice.ToString("c"), z.AvgTheaters, z.BestFilm}; 

results

index,Distributor,TotalBoxOffice,AvgTheaters,BestFilm
0,Disney,"$4,212,760,287.00",4351,Avengers: Endgame
1,Warner Bros.,"$1,695,953,630.00",3976,Joker
2,Sony Pictures Entertainment (SPE),"$1,503,714,590.00",3682,Spider-Man: Far from Home
3,Universal Pictures,"$1,298,633,470.00",3402,Us
4,Lionsgate,"$757,463,985.00",3181,John Wick: Chapter 3 - Parabellum
5,Twentieth Century Fox,"$692,033,142.00",3440,Bohemian Rhapsody
6,Paramount Pictures,"$632,860,971.00",3536,Bumblebee
7,STX Entertainment,"$267,377,638.00",3108,The Upside
8,United Artists Releasing,"$187,351,308.00",3177,The Addams Family
9,Focus Features,"$164,558,977.00",2564,Downton Abbey


## Extension Methods

https://docs.microsoft.com/dotnet/csharp/programming-guide/classes-and-structs/extension-methods?WT.mc_id=visualstudio-twitch-jefritz